In [ ]:
import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os

In [ ]:
SrcPath="/dog-breeds/"
imagePaths = sorted(list(paths.list_images(SrcPath)))
random.seed(42)
random.shuffle(imagePaths)
print(len(imagePaths))
print(imagePaths[0])

In [ ]:
from glob import glob

In [ ]:
src_path = glob("/dog-breeds/*/", recursive = True)

In [ ]:
src_path

In [ ]:
classes = []

for i in src_path:
    classes.append(i.split("/")[-2])

In [ ]:
classes

In [ ]:
#imagePaths
data = []
labels = []

In [ ]:
classes_dict = {}
count = 0
for i in classes :
    classes_dict[i] = count
    count+=1

In [ ]:
classes_dict

In [ ]:
print(imagePaths[0].split("/")[-2])

In [ ]:
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (224, 224))
    image = img_to_array(image)
    data.append(image)
    
    label = classes_dict[imagePath.split('/')[-2]]
    labels.append(label)
    

In [ ]:
len(data)

In [ ]:
len(labels)

In [ ]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

print(data.shape)
print(labels.shape)

In [ ]:
print(labels[1])

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)

In [ ]:
trainY = to_categorical(trainY, num_classes=8)
testY = to_categorical(testY, num_classes=8)

In [ ]:
labels[0]

In [ ]:
print(trainX.shape)
print(trainY.shape)

In [ ]:
print(trainY[4])

In [ ]:
print(testX.shape)
print(testY.shape)

In [ ]:
EPOCHS = 25
INIT_LR = 0.001
BS = 32
height=224
width=224
depth=3
numClasses=8

In [ ]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

model = Sequential()
inputShape = (height, width, depth)

# if we are using "channels first", update the input shape
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)

# first set of CONV => RELU => POOL layers
model.add(Conv2D(16, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# second set of CONV => RELU => POOL layers
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# third set of CONV => RELU => POOL layers
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))

# softmax classifier
model.add(Dense(numClasses))
model.add(Activation("softmax"))

In [ ]:
print(model.summary())

In [ ]:
import tensorflow as tfprint("[INFO] compiling model...")
opt = Adam(lr=INIT_LR)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt,metrics=["accuracy"])

In [ ]:
print("[INFO] training network...")
H = model.fit(trainX, trainY, batch_size=BS,validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,epochs=10)

In [ ]:
print("[INFO] serializing network...")
model.save("model.hd5", save_format="h5")